Imports needed to run the code

In [1]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from mewpy.simulation import get_simulator
from mewpy.simulation import set_default_solver
from mewpy.model import CommunityModel 
from os import listdir
from os.path import isfile, join
from math import inf

set_default_solver('cplex')

Creating a comprehension list with the models that will be used in the community model

In [2]:
FOLDER = 'models'
m = [f for f in listdir(FOLDER) if isfile(join(FOLDER, f))]

Read, rename and open all drains for each organism model

In [3]:
models = []
for f in m:
    
    # load the models
    model = read_sbml_model(join(FOLDER, f))
    
    # rename the models
    model.id = f[:-4]
    
    # open all exchange reactions
    sim = get_simulator(model)
    for r_id in sim.get_exchange_reactions():
        sim.set_reaction_bounds(r_id,-inf,inf)
        
    models.append(sim.model)
    

Build the community model 

In [4]:
cm = CommunityModel(models)

In [5]:
sim = cm.comm_model

Doing a pFBA to find a flux distribution that gives the optimal growth rate, while minimizing the total sum of the flux

In [6]:
res = sim.simulate(method='pFBA')
res

objective: 725724.1035309915
Status: OPTIMAL
Constraints: OrderedDict()
Method:pFBA

Observing the biomass of the community model

In [7]:
# community biomass
res.find(cm.biomass)

,Flux rate
Reaction ID,
community_growth,1316.52876


Observing the biomass of each organism

In [8]:
res.find(list(cm.biomasses.values()))

,Flux rate
Reaction ID,
biomass525_Klebsiella_pneumoniae_pneumoniae_MGH78578,107.795188
biomass166_Bacillus_subtilis_str_168,123.692103
biomass383_Kluyvera_ascorbata_ATCC_33433,157.498106
biomass525_Clostridium_sp_7_2_43FAA,91.462414
biomass345_Escherichia_coli_SE11,155.471927
biomass360_Morganella_morganii_subsp_morganii_KT,151.471276
biomass345_Bifidobacterium_longum_DJO10A,76.277167
biomass402_Klebsiella_oxytoca_KCTC_1686,157.596633
biomass386_Enterobacter_cloacae_EcWSU1,206.481381


Check the community model uptakes

In [39]:
up = [x[:-3] for x in sim.get_exchange_reactions()]

In [40]:
df.loc[df['lb']!=-inf]

,name,lb,ub,stoichiometry,gpr,Flux rate
id,,,,,,
community_growth,Community growth rate,0.0,inf,{'community_biomass': -1},,530.1671


Check all the reactions with a flux rate = 0

In [41]:
df=sim.find(up).join(res.find(up))
df.loc[df['Flux rate']==0]

,name,lb,ub,stoichiometry,gpr,Flux rate
id,,,,,,
EX_13ppd[e],EX_13ppd[e],-inf,inf,{'13ppd[e]': -1},,-0.0
EX_15dap[e],EX_15dap[e],-inf,inf,{'15dap[e]': -1},,-0.0
EX_2ddglcn[e],EX_2ddglcn[e],-inf,inf,{'2ddglcn[e]': -1},,0.0
EX_2hyoxplac[e],EX_2hyoxplac[e],-inf,inf,{'2hyoxplac[e]': -1},,-0.0
EX_34dhpha[e],EX_34dhpha[e],-inf,inf,{'34dhpha[e]': -1},,-0.0
...,...,...,...,...,...,...
EX_dextrin[e],EX_dextrin[e],-inf,inf,{'dextrin[e]': -1},,-0.0
EX_glctnb[e],EX_glctnb[e],-inf,inf,{'glctnb[e]': -1},,0.0
EX_thf[e],EX_thf[e],-inf,inf,{'thf[e]': -1},,-0.0


Since all uptakes are open the community growth should have been unbounded. 
The models used probably need to be curated.

Creation of a minimal medium containing only the essential reactions

In [27]:
from cobra.medium.minimal_medium import minimal_medium
minimal_mediums = dict()
for m in models:
    medium =minimal_medium(sim.model,min_objective_value=0.1).to_dict()
    minimal_mediums[m.id]=medium

Select the uptakes needed for the model to grow

In [32]:
uptakes= set()
for r_exs in minimal_mediums.values():
    uptakes = uptakes.union(set(r_exs.keys()))
uptakes

{'EX_MGlcn65[e]',
 'EX_MGlcn65_rl[e]',
 'EX_amp[e]',
 'EX_ca2[e]',
 'EX_cl[e]',
 'EX_cobalt2[e]',
 'EX_cu2[e]',
 'EX_fe2[e]',
 'EX_fe3[e]',
 'EX_k[e]',
 'EX_mg2[e]',
 'EX_mn2[e]',
 'EX_nmn[e]',
 'EX_pheme[e]',
 'EX_sheme[e]',
 'EX_so4[e]',
 'EX_tet[e]',
 'EX_thm[e]',
 'EX_zn2[e]'}

In [34]:
medium = {k:(0,inf) for k in sim.get_uptake_reactions()}
medium.update({k:(-inf,inf) for k in uptakes})

Runing a pFBA with the minimal medium obtained

In [37]:
res = sim.simulate(method='pFBA',constraints=medium)
res.find(list(cm.biomasses.values()))

,Flux rate
Reaction ID,
biomass525_Klebsiella_pneumoniae_pneumoniae_MGH78578,88.439737
biomass166_Bacillus_subtilis_str_168,0.000000
biomass383_Kluyvera_ascorbata_ATCC_33433,107.205988
biomass525_Clostridium_sp_7_2_43FAA,0.000000
biomass345_Escherichia_coli_SE11,83.103535
biomass360_Morganella_morganii_subsp_morganii_KT,79.283456
biomass345_Bifidobacterium_longum_DJO10A,0.000000
biomass402_Klebsiella_oxytoca_KCTC_1686,140.940773
biomass386_Enterobacter_cloacae_EcWSU1,118.059595


If we only allow for uptakes that are essential for each organisms individualy, some do not grow 

Organisms with faster metabolisms prevent other to grow

Code developed to enrich the medium

In [43]:
res = sim.simulate(method='pFBA')
print(res)
not_used = [x for x in sim.get_uptake_reactions() if res.fluxes[x]==0]
print('N. of external metabolites not used:',len(not_used))

objective: 725724.1035310331
Status: OPTIMAL
Constraints: OrderedDict()
Method:pFBA
N. of external metabolites not used: 561


Defining a minimal medium

In [44]:
m=minimal_medium(sim.model,min_objective_value=1.0).to_dict()
m

{'EX_ca2[e]': 0.0030965,
 'EX_cl[e]': 0.0030965,
 'EX_cobalt2[e]': 0.0030965,
 'EX_cu2[e]': 0.0030965,
 'EX_fe3[e]': 0.006193,
 'EX_k[e]': 0.0030965,
 'EX_mg2[e]': 0.0030965,
 'EX_mn2[e]': 0.0030965,
 'EX_nmn[e]': 0.30810650000177975,
 'EX_so4[e]': 0.0030965,
 'EX_thm[e]': 0.0030965,
 'EX_zn2[e]': 0.0030965,
 'EX_pheme[e]': 0.0030965,
 'EX_amp[e]': 0.5039670000015443,
 'EX_sheme[e]': 0.0030965,
 'EX_MGlcn65_rl[e]': 1.078158499999669,
 'EX_tet[e]': 0.054083000000019865}

Check if from the new medium there is any uptake that it's now used but previously wasn't

In [45]:
intersect = set(m.keys()).intersection(set(not_used))
intersect

{'EX_MGlcn65_rl[e]', 'EX_tet[e]'}

Adding the new uptakes to a list with other used ones

In [46]:
not_used = list(set(not_used)-intersect)
used = [x for x in sim.get_uptake_reactions() if x not in not_used]
print('N. of external metabolites used:',len(used))

N. of external metabolites used: 120


Runing pFBA

In [50]:
medium={x:0 for x in not_used}
medium.update({x:(-inf,inf) for x in used})

res = sim.simulate(method='pFBA',constraints=medium)
res.find(list(cm.biomasses.values()))

,Flux rate
Reaction ID,
biomass525_Klebsiella_pneumoniae_pneumoniae_MGH78578,107.795188
biomass166_Bacillus_subtilis_str_168,123.692103
biomass383_Kluyvera_ascorbata_ATCC_33433,157.498106
biomass525_Clostridium_sp_7_2_43FAA,91.462414
biomass345_Escherichia_coli_SE11,155.471927
biomass360_Morganella_morganii_subsp_morganii_KT,151.471276
biomass345_Bifidobacterium_longum_DJO10A,76.277167
biomass402_Klebsiella_oxytoca_KCTC_1686,157.596633
biomass386_Enterobacter_cloacae_EcWSU1,206.481381


Imports needed to make the optimization problem to find the best medium for the community model

In [61]:
from mewpy.problems import MediumProblem
from mewpy.optimization.evaluation import MolecularWeight, CNRFA, TargetFlux
import numpy as np

Creating the optimization problem

In [63]:
# list with the uptake reactions
uptakes = used

# list with the biomasses of the organisms
biomasses = list(cm.biomasses.values())

# Optimization Objectives:
##########################

# Minimize the uptakes of the Molecular Weight
f1 = MolecularWeight(uptakes,maximize=False)

# Maximize the number of organisms growing above 0.1 1/h
f2 = CNRFA(biomasses,maximize=True)

# Minimize the community growth
f3 = TargetFlux(cm.biomass,maximize=False)


# close all uptakes
for rx in sim.get_uptake_reactions():
    sim.set_reaction_bounds(rx,0,0)

n = len(uptakes)

# Possible maximum uptake rates in [0,10,20,30,...,300]
levels = np.linspace(0, 300, 31)


problem = MediumProblem(sim.model,
                        [f1,f2,f3],
                        candidate_min_size=n, 
                        candidate_max_size=n, 
                        target=uptakes,
                        levels = levels
                       )

Runing the optimization problem with the help of Evolutionary Algorithm (EA)

In [65]:
from mewpy.optimization import EA

ea = EA(problem)
# solutions = ea.run()

Read the result of the EA

In [100]:
import pandas as pd
df = pd.read_csv('medium.csv',index_col=0)
df

,Modification,Size,Molecular Weight,CNRFA,TargetFlux
ID,,,,,
1,"{'EX_galct_D[e]': 30.0, 'EX_cl[e]': 70.0, 'EX_...",83,1907.006040,10,686.601857
26,"{'EX_galct_D[e]': 30.0, 'EX_dcyt[e]': 10.0, 'E...",90,2214.558517,10,705.815107
35,"{'EX_cl[e]': 50.0, 'EX_gthrd[e]': 290.0, 'EX_f...",94,2527.907592,10,714.505771
57,"{'EX_cl[e]': 70.0, 'EX_gthrd[e]': 290.0, 'EX_f...",94,2672.459385,10,723.166452
36,"{'EX_galct_D[e]': 30.0, 'EX_cl[e]': 70.0, 'EX_...",87,2687.744001,10,723.817187
39,"{'EX_cl[e]': 70.0, 'EX_fol[e]': 30.0, 'EX_ser_...",96,2905.641341,10,739.893197
2,"{'EX_nmn[e]': 240.0, 'EX_ppa[e]': 70.0, 'EX_gl...",112,3325.798791,10,538.235154
25,"{'EX_cl[e]': 70.0, 'EX_ppi[e]': 270.0, 'EX_ac[...",107,3443.949736,10,744.846974
22,"{'EX_cl[e]': 50.0, 'EX_acac[e]': 220.0, 'EX_ma...",117,3455.974329,10,760.076793


Selecting the best medium to use for the modelling

In [104]:
solution = eval(df.loc[1]['Modification'])
solution

{'EX_galct_D[e]': 30.0,
 'EX_cl[e]': 70.0,
 'EX_ser_L[e]': 270.0,
 'EX_dcyt[e]': 10.0,
 'EX_sucr[e]': 0.0,
 'EX_nmn[e]': 230.0,
 'EX_trp_L[e]': 30.0,
 'EX_lac_L[e]': 90.0,
 'EX_mal_L[e]': 0.0,
 'EX_fol[e]': 290.0,
 'EX_cit[e]': 50.0,
 'EX_cu2[e]': 120.0,
 'EX_fe2[e]': 180.0,
 'EX_alahis[e]': 150.0,
 'EX_nac[e]': 60.0,
 'EX_oxa[e]': 170.0,
 'EX_tsul[e]': 70.0,
 'EX_pnto_R[e]': 190.0,
 'EX_k[e]': 20.0,
 'EX_ppa[e]': 230.0,
 'EX_orn[e]': 300.0,
 'EX_na1[e]': 80.0,
 'EX_o2[e]': 30.0,
 'EX_ac[e]': 110.0,
 'EX_fe3[e]': 100.0,
 'EX_duri[e]': 50.0,
 'EX_xan[e]': 260.0,
 'EX_4abz[e]': 200.0,
 'EX_asn_L[e]': 240.0,
 'EX_mg2[e]': 60.0,
 'EX_sel[e]': 230.0,
 'EX_val_L[e]': 150.0,
 'EX_dgsn[e]': 90.0,
 'EX_glypro[e]': 60.0,
 'EX_mqn7[e]': 80.0,
 'EX_g6p[e]': 70.0,
 'EX_pheme[e]': 280.0,
 'EX_26dap_M[e]': 0.0,
 'EX_so4[e]': 270.0,
 'EX_glyasn[e]': 270.0,
 'EX_MGlcn181[e]': 210.0,
 'EX_thymd[e]': 180.0,
 'EX_cytd[e]': 160.0,
 'EX_arg_L[e]': 50.0,
 'EX_tyr_L[e]': 70.0,
 'EX_ca2[e]': 270.0,
 'EX_cobalt

Changing the format of the uptakes

In [107]:
const = {k:(0,inf) for k in sim.get_uptake_reactions()}
const.update({k : (-v,1000) for k,v in solution.items()})

Doing a pFBA with medium from the EA

In [108]:
res = sim.simulate(method='pFBA',constraints=const)
res.find(list(cm.biomasses.values()))

,Flux rate
Reaction ID,
biomass525_Klebsiella_pneumoniae_pneumoniae_MGH78578,0.000000
biomass166_Bacillus_subtilis_str_168,8.474965
biomass383_Kluyvera_ascorbata_ATCC_33433,110.382282
biomass525_Clostridium_sp_7_2_43FAA,20.137757
biomass345_Escherichia_coli_SE11,45.121717
biomass360_Morganella_morganii_subsp_morganii_KT,0.000000
biomass345_Bifidobacterium_longum_DJO10A,0.000000
biomass402_Klebsiella_oxytoca_KCTC_1686,56.528968
biomass386_Enterobacter_cloacae_EcWSU1,141.326013


Comand SteadyCom implemented in MEWpy predict changes in the species abundance

The medium in this first simulation a open one (batch)

In [97]:
from mewpy.cobra.steadycom import SteadyCom
SteadyCom(cm)

Community growth: 206.48046874999997
Klebsiella_pneumoniae_pneumoniae_MGH78578	0.0
Bacillus_subtilis_str_168	0.0
Kluyvera_ascorbata_ATCC_33433	0.0
Clostridium_sp_7_2_43FAA	0.0
Escherichia_coli_SE11	0.0
Morganella_morganii_subsp_morganii_KT	0.0
Bifidobacterium_longum_DJO10A	0.0
Klebsiella_oxytoca_KCTC_1686	0.0
Enterobacter_cloacae_EcWSU1	1.0
Enterococcus_faecalis_TX0104	0.0

In batch medium the Enterobacter cloacae would take over all other organisms

Comand SteadyCom but this time we use the menimal medium obtained from the optimization problem

In [98]:
SteadyCom(cm,constraints=const)

Community growth: 103.96289062499997
Klebsiella_pneumoniae_pneumoniae_MGH78578	0.036119878517129465
Bacillus_subtilis_str_168	0.0
Kluyvera_ascorbata_ATCC_33433	0.15093323732430652
Clostridium_sp_7_2_43FAA	0.0
Escherichia_coli_SE11	0.316151197866958
Morganella_morganii_subsp_morganii_KT	0.0
Bifidobacterium_longum_DJO10A	0.0
Klebsiella_oxytoca_KCTC_1686	0.25611987851712953
Enterobacter_cloacae_EcWSU1	0.2406758077744766
Enterococcus_faecalis_TX0104	0.0